In [1]:
import pandas as pd
import numpy as np

pd.set_option('max.columns', 131)

# Resultados do Active Learning

In [2]:
df1 = pd.read_csv('raw_data_with_labels.csv')
df1 = df1[df1['y'].notnull()]
df1.shape

(91, 4)

In [3]:
df2 = pd.read_csv('active_label1_done.csv', index_col=0)
df2 = df2[df2['y'].notnull()]
df2['new'] = 1
df2.shape

(102, 6)

In [4]:
df2.head(1)

,title,upload_date,view_count,y,p,new
45,Deep Learning at Scale with Horovod feat. Trav...,2021-01-28,149,0,0.404861,1


In [5]:
from sklearn.metrics import roc_auc_score, average_precision_score
average_precision_score(df2['y'], df2['p']), roc_auc_score(df2['y'], df2['p'])

(0.3838108380590924, 0.5236673773987206)

In [6]:
df = pd.concat([df1, df2.drop('p', axis=1)])

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [35]:
df['upload_date'] = pd.to_datetime(df['upload_date'], format='%Y-%m-%d')
df['view_count'] = df['view_count'].astype(int)
df = df.sort_values('upload_date')
df_limpo = df.copy()
df_limpo['new'] = df_limpo['new'].fillna(0)
df_limpo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193 entries, 294 to 0
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   title        193 non-null    object        
 1   upload_date  193 non-null    datetime64[ns]
 2   view_count   193 non-null    int64         
 3   y            193 non-null    float64       
 4   new          193 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 9.0+ KB


In [36]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()

In [37]:
features['tempo_desde_pub'] = (pd.to_datetime('2021-01-31') - df_limpo['upload_date']) / np.timedelta64(1, 'D')
features['views'] = df_limpo['view_count']
features['views_por_dia'] = features['views'] / features['tempo_desde_pub']
features = features.drop(['tempo_desde_pub'], axis=1)

In [63]:
mask_train = df_limpo['new'] == 0

Xtrain, Xval = features.iloc[:130][mask_train], features.iloc[130:]
ytrain, yval = y.iloc[:130][mask_train], y.iloc[130:]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

<ipython-input-63-adee706cedae>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Xtrain, Xval = features.iloc[:130][mask_train], features.iloc[130:]


((42, 2), (63, 2), (42,), (63,))

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_train = df_limpo.iloc[:130][mask_train]['title']
title_val = df_limpo.iloc[130:]['title']

title_vec = TfidfVectorizer(min_df=2)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

<ipython-input-45-4f966e4c0d61>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  title_train = df_limpo.iloc[:130][mask_train]['title']


In [46]:
title_bow_train.shape

(42, 43)

In [47]:
from scipy.sparse import hstack

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [48]:
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, class_weight='balanced', n_jobs=3)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=3,
                       random_state=0)

In [49]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

In [50]:
average_precision_score(yval, p)

0.36862917382030524

In [51]:
roc_auc_score(yval, p)

0.5406976744186047

# Aumentando o treino

In [64]:
#mask_val = df_limpo['new'] == 0

Xtrain, Xval = features.iloc[:130], features.iloc[130:]
ytrain, yval = y.iloc[:130], y.iloc[130:]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((130, 2), (63, 2), (130,), (63,))

In [65]:
title_train = df_limpo.iloc[:130]['title']
title_val = df_limpo.iloc[130:]['title']

title_vec = TfidfVectorizer(min_df=2)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

In [66]:
Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [67]:
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, class_weight='balanced', n_jobs=3)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=3,
                       random_state=0)

In [68]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

In [69]:
average_precision_score(yval, p)

0.38044944752795073

In [70]:
roc_auc_score(yval, p)

0.5354651162790698